In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import warnings
import string
import re

warnings.filterwarnings('ignore')
plt.style.use('ggplot')

In [2]:
francetax = pd.read_parquet('data/france_revenue_tax_syndicates.parquet')

In [3]:
francetax.head()

,COMMUNE,Libellé commune,ANNEE,TFNB,TFB,TH,SYNDICATE_TFNB,SYNDICATE_TFB,SYNDICATE_TH,TP,POPULATION
0,1,ABERGEMENT CLEMENCIAT,2022,28443.0,144513.0,18963.0,0.0,0.0,0.0,0.0,798.0
1,2,ABERGEMENT DE VAREY,2022,788.0,61501.0,14164.0,0.0,0.0,0.0,0.0,257.0
2,4,AMBERIEU EN BUGEY,2022,23997.0,7483843.0,387180.0,0.0,0.0,0.0,0.0,14514.0
3,5,AMBERIEUX EN DOMBES,2022,30661.0,499270.0,54125.0,0.0,0.0,0.0,0.0,1776.0
4,6,AMBLEON,2022,3001.0,25437.0,6739.0,0.0,0.0,0.0,0.0,118.0


In [4]:

# Function to standardize municipality names
def standardize_name(name):
    return ''.join(char for char in name if char not in string.punctuation).strip().upper()

# Apply the function to the 'Libellé commune' column
francetax['Libellé commune'] = francetax['Libellé commune'].apply(standardize_name)

# Convert columns to appropriate data types
francetax[['TFNB', 'TFB', 'TH', 'SYNDICATE_TFNB','SYNDICATE_TFB','SYNDICATE_TH']] = francetax[['TFNB', 'TFB', 'TH', 'SYNDICATE_TFNB','SYNDICATE_TFB','SYNDICATE_TH']].apply(pd.to_numeric, errors='coerce')

# Create a complete range of years for each municipality
years = np.arange(1982, 2023)
all_municipalities = francetax['Libellé commune'].unique()

# Create a complete DataFrame with all municipalities and all years
complete_data = pd.MultiIndex.from_product([all_municipalities, years], names=['Libellé commune', 'ANNEE']).to_frame(index=False)

# Merge with the existing data
data_complete = pd.merge(complete_data, francetax, on=['Libellé commune', 'ANNEE'], how='left')

In [5]:
# Function to fill missing data for each municipality
def fill_missing_data(df):
    for col in ['TFNB', 'TFB', 'TH', 'SYNDICATE_TFNB','SYNDICATE_TFB','SYNDICATE_TH']:
        df[col] = df[col].fillna(method='ffill')  # Fill using data from the previous year
        if df[col].isna().sum() > 0:
            df[col] = df[col].fillna(df[col].rolling(window=5, min_periods=1).mean())  # Fill remaining using the mean of the previous years
    return df

# Apply the function to each municipality
data_complete = data_complete.groupby('Libellé commune').apply(fill_missing_data).reset_index(drop=True)


In [6]:
# Conversion rate from francs to euros
conversion_rate = 6.55957

# Identify rows where year is before 2002
mask = data_complete['ANNEE'] < 2002

# Apply conversion to the tax columns
data_complete.loc[mask, 'TFNB'] = data_complete.loc[mask, 'TFNB'] / conversion_rate
data_complete.loc[mask, 'TFB'] = data_complete.loc[mask, 'TFB'] / conversion_rate
data_complete.loc[mask, 'TH'] = data_complete.loc[mask, 'TH'] / conversion_rate
data_complete.loc[mask, 'SYNDICATE_TFNB'] = data_complete.loc[mask, 'SYNDICATE_TFNB'] / conversion_rate
data_complete.loc[mask, 'SYNDICATE_TFB'] = data_complete.loc[mask, 'SYNDICATE_TFB'] / conversion_rate
data_complete.loc[mask, 'SYNDICATE_TH'] = data_complete.loc[mask, 'SYNDICATE_TH'] / conversion_rate


In [7]:
def impute_taxes(group):
    for col in ['TFNB', 'TFB', 'TH', 'SYNDICATE_TFNB','SYNDICATE_TFB','SYNDICATE_TH']:
        values = group[col]
        
        for i in range(len(values)):
            if pd.isna(values.iloc[i]):
                # Get the previous 5 years
                previous_values = values.iloc[max(0, i-5):i]
                
                if not previous_values.isna().all():
                    # If there are previous values, use the mean or median
                    if len(previous_values.dropna()) > 0:
                        values.iloc[i] = previous_values.mean()
                    else:
                        values.iloc[i] = previous_values.median()
                else:
                    # If all upcoming years are NaN, impute 0
                    values.iloc[i] = 0
    
    return group


# App#ly the imputation function to each municipality group
data = data_complete.groupby('Libellé commune').apply(impute_taxes)
data.head()


,Libellé commune,ANNEE,COMMUNE,TFNB,TFB,TH,SYNDICATE_TFNB,SYNDICATE_TFB,SYNDICATE_TH,TP,POPULATION
0,ABERGEMENT CLEMENCIAT,1982,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1,ABERGEMENT CLEMENCIAT,1983,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2,ABERGEMENT CLEMENCIAT,1984,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
3,ABERGEMENT CLEMENCIAT,1985,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4,ABERGEMENT CLEMENCIAT,1986,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [8]:
def standardize_name(name):
    return re.sub(r'\W+', '', name).strip().upper()  # Remove non-alphanumeric characters and convert to uppercase

# Create a mapping from standardized names to original names
data['Standardized Name'] = data['Libellé commune'].apply(standardize_name)
name_mapping = data.drop_duplicates('Standardized Name')[['Standardized Name', 'Libellé commune']].set_index('Standardized Name')['Libellé commune'].to_dict()

# Use the standardized names in the main data
data['Libellé commune'] = data['Standardized Name']
data = data.drop(columns=['Standardized Name'])

#SAVE CLEANED DATA
data.to_parquet('data/francetax_processed_syndicates.parquet')

In [14]:
# Calculate the total amount of each tax for municipalities
municipal_totals = data[['TFNB', 'TFB', 'TH']].sum()
most_significant_municipal_tax = municipal_totals.idxmax()

# Calculate the total amount of each tax for labor unions (syndicates)
syndicate_totals = data[['SYNDICATE_TFNB', 'SYNDICATE_TFB', 'SYNDICATE_TH']].sum()
most_significant_syndicate_tax = syndicate_totals.idxmax()

# Compare the predominant tax for municipalities and labor unions
same_tax_predominant = most_significant_municipal_tax == most_significant_syndicate_tax

# Results
print(f"Most significant tax for municipalities: {most_significant_municipal_tax}")
print(f"Most significant tax for labor unions: {most_significant_syndicate_tax}")


Most significant tax for municipalities: TFB
Most significant tax for labor unions: SYNDICATE_TFB


In [15]:
municipal_totals

TFNB    5.454728e+10
TFB     6.541395e+11
TH      5.144024e+11
dtype: float64

In [16]:
syndicate_totals

SYNDICATE_TFNB    8.788824e+08
SYNDICATE_TFB     6.545035e+09
SYNDICATE_TH      5.674741e+09
dtype: float64